In [4]:
# import
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer

from konlpy.tag import Okt
from tqdm import tqdm
import pandas as pd

# DB접속
import pymysql
import numpy as np
from glob import glob
import os 
from google.cloud import bigquery

import re
import matplotlib.pyplot as plt

In [5]:
def listToString(s):  
    str1 = ""  
    for ele in s:  
        str1 += " " + ele.strip()  
    return str1

In [6]:
def clean_str(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    pattern = '[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]'        # 특수기호제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    text = text.replace('\n', '')
    text = text.replace('\r', '')
    return text

In [8]:
category_group   = '오트'
category_eng     = 'ottmilk'
category_keyword ='어메이징'

In [9]:
#########
# 2.자료 추출 : BigQuery
###

# 접속 정보 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./vision API-06a448b64428.json"
client = bigquery.Client()

# 쿼리실행
sql = """SELECT USER
         , FORMAT_DATETIME('%Y-%m-%d', CAST(A.DATE AS DATETIME)) as REG_DTM
         , FORMAT_DATETIME('%Y-%m', CAST(A.DATE AS DATETIME)) as REG_DT
         , FORMAT_DATETIME('%YY-%VW', CAST(A.DATE AS DATETIME)) as REG_WEEK
         , CHANNEL
         , PRODUCT
         , REVIEW
         , SCORE
         FROM `thermal-rain-234004.review.review_all` A
         where PRODUCT like '%"""+category_group+"""%'
         and PRODUCT like '%"""+category_keyword+"""%'
         and CAST(A.DATE AS DATETIME) between DATE_SUB(current_date(), INTERVAL 90 DAY) and DATE_SUB(current_date(), INTERVAL -1 DAY)
         order by CAST(A.DATE AS DATETIME) desc"""
df1 = client.query(sql).to_dataframe()
df1 = df1.replace('kakaomakers', '카카오 선물하기')

In [10]:
df1 = df1.replace('어메이징 오트 언스위트 190ml 48팩', '어메이징 오트 [ 언스위트 ] [ 190ML ]')
df1 = df1.replace('어메이징 오트 언스위트 190ml 24팩', '어메이징 오트 [ 언스위트 ] [ 190ML ]')
df1 = df1.replace('어메이징 오트 오리지널 190ml 24팩', '어메이징 오트 [ 오리지널 ] [ 190ML ]')
df1 = df1.replace('어메이징 오트 오리지널 190ml 48팩', '어메이징 오트 [ 오리지널 ] [ 190ML ]')
df1 = df1.replace('매일유업 어메이징 오트 190ml x 24팩 (오리지널/언스위트)', '어메이징 오트 [ 언스위트 ] [ 190ML ],어메이징 오트 [ 오리지널 ] [ 190ML ]')
df1 = df1.replace('[매일유업] 어메이징 오트 오리지널 / 언스위트 190ml 24팩', '어메이징 오트 [ 언스위트 ] [ 190ML ],어메이징 오트 [ 오리지널 ] [ 190ML ]')
df1 = df1.replace('매일 두유 6종 / (신제품)어메이징오트 2종 190ml 24팩 택1', '매일 두유 6종 / 어메이징오트 2종 190ml 24팩 택1')

df1 = df1.replace('매일유업 어메이징 오트 바리스타 950ml 3팩',         '어메이징 오트 [ 바리스타 ] [ 950ML ]')
df1 = df1.replace('[매일유업] 어메이징오트 바리스타 950ml x 3팩',      '어메이징 오트 [ 바리스타 ] [ 950ML ]')
df1 = df1.replace('매일유업 어메이징 오트 언스위트 두유, 24개, 190ml', '어메이징 오트 [ 언스위트 ] [ 190ML ]')
df1 = df1.replace('매일유업 어메이징 오트 오리지널 두유, 24개, 190ml', '어메이징 오트 [ 오리지널 ] [ 190ML ]')

df1 = df1.replace('선택: 어메이징 오트 오리지널', '어메이징 오트 [ 오리지널 ] [ 190ML ]')
df1 = df1.replace('(대용량) 아몬드브리즈/매일두유/어메이징오트 950ml x 10팩', '어메이징 오트 [ 바리스타 ] [ 950ML ]')

In [11]:
###############
# 2.자료 추출 : MySQL
#####
# 접속 정보 
conn = pymysql.connect(host = '10.223.7.4', user ='MAEIL_CS', password = "Maeil01!@", database='MAEIL_CS')
cursor = conn.cursor(pymysql.cursors.DictCursor)

# 쿼리실행: 전체리뷰
sql= '''select A.WRITER as USER
        , (STR_TO_DATE(A.WRITE_DT, '%Y%m%d'))  as REG_DTM 
        , left((STR_TO_DATE(A.WRITE_DT, '%Y%m%d')),7)  as REG_DT 
        , DATE_FORMAT(STR_TO_DATE(A.WRITE_DT, '%Y%m%d'),'%YY-%uW' )  as REG_WEEK 
        , (SELECT CODE_NM FROM JT_CODE B WHERE A.COMPANY_CODE = B.CODE AND B.CODE_GRP_ID = 'DEALER_ID') as CHANNEL 
        , group_concat(distinct B.CODE_NM order by CODE_NM asc) as PRODUCT
        , A.GRADE as SCORE
        , trim(A.CONTENTS) as REVIEW
        FROM MAEIL_REVIEW A, JT_CODE B
        where A.GRADE in (1,2,3,4,5)
        and CAST(STR_TO_DATE(A.WRITE_DT, '%Y%m%d') AS DATETIME) between DATE_SUB(current_date(), INTERVAL 90 DAY) and DATE_SUB(current_date(), INTERVAL -1 DAY)
        and A.PRODUCT_CODE = B.CODE 
        AND B.CODE_NM like '%'''+category_group +'''%'
        AND B.CODE_NM like '%'''+category_keyword +'''%'
        and B.CODE not like'G_%'
        group by A.WRITER, A.WRITE_DT, A.COMPANY_CODE, GRADE, CONTENTS
        order by A.WRITE_DT desc
        '''
cursor.execute(sql)
df2 = cursor.fetchall()
df2 = pd.DataFrame(df2)

df2 = df2.replace('25 카카오 톡스토어(20.12.2~미사용)', '25 카카오톡')

In [12]:
df = pd.concat([df1, df2], ignore_index=True)
df['REG_DTM'] = pd.to_datetime(df['REG_DTM'],format = '%Y-%m-%d')
# df['REG_DTM'] = df['REG_DTM']
df['SCORE'] = df['SCORE'].astype(float)
df['REVIEW'] = df['REVIEW'].str.replace('\n', ' ')
df['lenght'] = df['REVIEW'].str.len()

df = df[df.lenght != 0]
df = df.reset_index(drop=True)

In [13]:
documents =list(df.REVIEW)

In [14]:
documents_temp = []

for text in documents:
    documents_temp.append(clean_str(text))

In [15]:
documents_temp[:10]

['너무 맛잇어서 다시시켰어요  아빠 드리니까 회사가져가서 드셔보시더니 시키셨슴다 ㅋㅋㅋㅋ 그냥 먹어도 고소하고 맛있어요 바리스타맛이 저는 오리지널보다 좀 더 고소하고 부드러운듯 그리고 콜드브루랑 타먹는거 진짜 추천 약간 토피넛 라떼 향도 나고 고소한 곡물 라떼 같기도 하고 암튼 또 살거임',
 '메이커스 두부 미숫가루와 함께 먹고 있는데 궁합이 아주 좋아요  맛이있어서 재구매 의사 있습니다 ',
 '고소하고 맛있어요 ',
 '맛있어요   이렇게 먹어도 맛있고 저렇게 먹어도 맛있고 ㅎㅎ 또 오픈해주세요  ',
 '맛이 진하면서 달지도않고 담백하니 맛있어요  ',
 '기본적으로 달달한 맛이 있어서 드립커피와 가장 잘 어울리는 강점이 있네요  그냥 마셔보기도 했는데 역시 설명처럼 커피를 더욱 부드럽게 만들어주니까 우유가 부담스러웠던 저에게는 더욱 반가운 제품이었습니다  개인적으로는 단맛을 줄여서 좀더 담백하게 출시되어도 괜찮겠다 싶어요  왜냐면 단거랑 같이 먹고싶거든요 히히히',
 '잘받았습니다',
 '정말 강추합니다 일리 에스프레소 내려서 오트우유와 믹스하면 찰떡궁합입니다 또 구매할 의향있습니다  ',
 '담백하니 그냥 먹어도 맛있어요',
 '부담없고 정말 맛있고  좋습니다 ']

In [16]:
preprocessed_documents = []

for line in tqdm(documents_temp):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)

100%|████████████████████████████████████████████████████████████████████████| 10212/10212 [00:00<00:00, 464200.37it/s]


In [17]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [18]:
custom_tokenizer = CustomTokenizer(Okt())
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=30000)
train_bow_embeddings = vectorizer.fit_transform(preprocessed_documents)

In [19]:
print(train_bow_embeddings.shape)

(10212, 10947)


In [20]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

C:\Users\MAEIL\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [21]:
id2token

{0: '06',
 1: '07',
 2: '09',
 3: '0906',
 4: '10',
 5: '100',
 6: '1000',
 7: '10000',
 8: '101',
 9: '10900원',
 10: '10분',
 11: '1111111',
 12: '11일',
 13: '12',
 14: '1200원',
 15: '12시',
 16: '1300원',
 17: '13일',
 18: '14',
 19: '145',
 20: '14일',
 21: '15',
 22: '150',
 23: '16',
 24: '17',
 25: '170',
 26: '17일',
 27: '180',
 28: '19',
 29: '190',
 30: '1년',
 31: '1등',
 32: '1분',
 33: '1억',
 34: '1일',
 35: '20',
 36: '200',
 37: '2022',
 38: '2022년',
 39: '21일',
 40: '22',
 41: '220906',
 42: '22년',
 43: '22일',
 44: '23일',
 45: '24',
 46: '24시간',
 47: '24일',
 48: '25',
 49: '26',
 50: '26일',
 51: '27',
 52: '27일',
 53: '28',
 54: '29',
 55: '29일',
 56: '2년',
 57: '2등',
 58: '2월',
 59: '2일',
 60: '2조',
 61: '30',
 62: '300',
 63: '3000원',
 64: '30분',
 65: '30초',
 66: '31',
 67: '3300원',
 68: '355',
 69: '365일',
 70: '3700원',
 71: '3등',
 72: '3월',
 73: '3일',
 74: '3천원',
 75: '40',
 76: '45',
 77: '48',
 78: '4시',
 79: '4일',
 80: '50',
 81: '500',
 82: '5600원',
 83: '5월',
 84: '5일',


In [22]:
vocab[:2]

['06', '07']

### LDA 학습하기

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [24]:
# TF-IDF 행렬로 변환
X = vectorizer.fit_transform(preprocessed_documents)

In [25]:
lda_model = LatentDirichletAllocation(n_components=20,learning_method='online',random_state=777,max_iter=1)
lda_top = lda_model.fit_transform(X)

In [26]:
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=7):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i]) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

Topic 1: ['이제', '신제품', '에서', '있고', '가격', '추가', '모르겠어요']
Topic 2: ['생각', '빨대', '보다', '종이', '처음', '인데', '있습니다']
Topic 3: ['감사합니다', '배송', '제품', '좋은', '이에요', '건강한', '빠른']
Topic 4: ['커피', '먹으니', '라떼', '맛있어요', '넣어', '에스프레소', '괜찮아요']
Topic 5: ['자주', '어울려요', '커피', '먹고있어요', '않아요', '마시다가', '좋아하는데']
Topic 6: ['커피', '주문', '맛있네요', '너무', '해서', '같아요', '많이']
Topic 7: ['배송', '받았습니다', '도착', '사서', '는데', '빨라요', '거품']
Topic 8: ['유통', '기한', '했어요', '만족합니다', '7월', '진한', '넉넉하고']
Topic 9: ['아몬드', '포장', '브리', '있어요', '맛있습니다', '먹다가', '먹고']
Topic 10: ['ㅋㅋ', '항상', '에요', '궁금해서', '마시니', '빠르게', '받았어요']
Topic 11: ['으로', '아침', '좋아요', '대용', '먹기', '바로', '대체']
Topic 12: ['같아요', '합니다', '오트밀', '다른', '맛있고', '마셔도', '용량']
Topic 13: ['배송', '빠르고', '구입', '했어요', '구매', '가격', '왔어요']
Topic 14: ['재구매', '그래도', '속이', '먹겠습니다', '되네요', '믿고', '예정']
Topic 15: ['라떼', '우유', '맛있어요', '커피', '고소하고', '보다', '어메이징']
Topic 16: ['좋고', '좋겠어요', '즐겨', '먹으니까', '단백질', '쉐이크', '애용']
Topic 17: ['ㅠㅠ', '없어요', '좋아서', '매번', '거의', '가볍고', '저번']
Topic 18: ['먹어', '좋아요',

### KoBERT 학습하기

In [72]:
from bertopic import BERTopic

ModuleNotFoundError: No module named 'bertopic'

### Combined TM 학습하기

In [27]:
train_contextualized_embeddings = bert_embeddings_from_list(preprocessed_documents, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Batches:   0%|          | 0/52 [00:00<?, ?it/s]

In [28]:
qt = TopicModelDataPreparation()
training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

In [29]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=20, num_epochs=20)
ctm.fit(training_dataset)

Epoch: [20/20]	 Seen Samples: [204240/204240]	Train Loss: 103.9083550315346	Time: 0:01:28.771621: : 20it [30:05, 90.28s/it] 
Sampling: [20/20]: : 20it [12:38, 37.94s/it]


In [30]:
# 1줄로 나오는 게 토픽당 7개 키워드라서...
ctm.get_topics(7)

defaultdict(list,
            {0: ['맛있어요', '먹으니', '맛있습니다', '먹으니까', '받지', '나고', '오트향'],
             1: ['먹기', '아침', '먹고', '대용', '간식', '부담', '하나'],
             2: ['업체', '부드러워서', '엄두', '짓이긴', '거슬리지', '용인', '돌아가면서'],
             3: ['하겠습니다', '싸게', '빠르고', '빠르네요', '17', '실온', '에겐'],
             4: ['라떼', '커피', '바리스타', '오트', '어메이징', '그냥', '에서'],
             5: ['빨대', '종이', '주문', '해서', '아몬드', '생각', '두유'],
             6: ['우유', '두유', '라떼', '오트밀', '귀리', '먹어', '아몬드'],
             7: ['보진', '무맛', '봤지만', '달콤함이', '않았지만', '쌓여있어서', '않아'],
             8: ['먹어도', '그냥', '에스프레소', '라떼', '커피', '보다', '넣어'],
             9: ['좋아요', 'lt', '사게', '시켜서', '않는데', '안되서', 'gt'],
             10: ['먹어도', '맛있네요', '만들어', '먹으니', '부드럽고', '넣어', '커피'],
             11: ['말모', '넣었는데', '높지만', '사봐야겟어요', '엄두', '순위', '놀라에'],
             12: ['우유', '두유', '아몬드', '보다', '으로', '생각', '귀리'],
             13: ['포장', '배송', '기한', '박스', '유통', '구매', '사은'],
             14: ['맛있어서', '주문', '했어요', '구매', '먹다가', '보고', '아침'],
          

### ZeroShotTM 학습

In [36]:
from contextualized_topic_models.models.ctm import ZeroShotTM

In [32]:
ctm2 = ZeroShotTM(bow_size=len(vocab), contextual_size=768, n_components=20, num_epochs=20)
ctm2.fit(training_dataset) # run the model

Epoch: [20/20]	 Seen Samples: [204240/204240]	Train Loss: 103.68092914585782	Time: 0:01:00.871437: : 20it [18:15, 54.79s/it]
Sampling: [20/20]: : 20it [09:11, 27.57s/it]


In [35]:
# 1줄로 나오는 게 토픽당 7개 키워드라서...
ctm2.get_topic_lists(7)

[['나더라구요', '다닐', '에겐', '했더니', '반해', '먹어보다', '뜯어서'],
 ['좋아해요', '맛있게', '먹어요', '만해', '아이', '먹고있어요', '부모님'],
 ['감사합니다', '빨라요', '빠른', '빠르고', '오고', '빠르게', '넉넉한'],
 ['커피', '라떼', '그냥', '바리스타', '우유', '어메이징', '오트'],
 ['포장', '제품', '박스', '기한', '유통', '사은', '구매'],
 ['종이', '빨대', '먹다가', '처음', '주문', '위트', '브리'],
 ['좋아요', '락토', '프리', '했는데요', '맞는데', '에게는', '나더라구요'],
 ['아침', '아이', '먹고', '다이어트', '간식', '먹기', '하나'],
 ['우유', '두유', '에서', '으로', '라떼', '오트밀', '아몬드'],
 ['맛있어요', '어울립니다', '이라는', '풍부해지고', '나름', '담백하니', '먹었더니'],
 ['부드럽고', '먹어도', '괜찮아요', '그냥', '고소하고', '넣어', '않아서'],
 ['먹으니', '에스프레소', '고소하고', '넣어', '먹어도', '고소해요', '커피'],
 ['빠른', '감사합니다', '빠르고', '받았습니다', '넉넉하고', '깔끔하게', '드립니다'],
 ['토피', '그렁', '나더라구요', '끝나', '맛있어용', '해봤네요', '트드'],
 ['바리스타', '해서', '으로', '어메이징', '아몬드', '주문', '보다'],
 ['샀어요', '했어요', '행사', '부모님', '드렸어요', '저렴하게', '선물'],
 ['우유', '라떼', '오트밀', '두유', '어메이징', '바리스타', '에서'],
 ['나오나', '친정', '세번', '나왔네요', '끝나', '달착지근한', '불가'],
 ['않고', '먹기', '속이', '편하고', '먹을', '부담', '밍밍'],
 ['종이', '빨대', '브리', '아몬드', '주문',

In [71]:
# import pyLDAvis as vis

# lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

# ctm_pd = vis.prepare(**lda_vis_data)
# vis.display(ctm_pd)